<a href="https://colab.research.google.com/github/MashaKubyshina/solving_work_data_analytics_problems/blob/main/Metircs_Early_Voting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Quick GOTV data from FB ads**

Goal of this tempalte is to facilitate reporting of GOTV (get out to vote) metrics. This script allows to pull a customized report with key GOTV metrics from 2 different data scources in less than 5 minutes. It used to take my team around 2 hours to pull this report by using excel and going through each data sheet manually.

This template can be used for any state. In this template I am using the state of Georgia.

**Key questions it will help you answer**

*   This report will answer questions such as:
*   What is the number of voting response engagements?
*   What percent of users clicking on our ads have already voted?
*   How many users in our audience will share the voting message with their family and friends on FB?

**What you need before running the code**

To use this template you will need the following data sheets downloaded on your machine:

*   Chatfuel data in csv (select the user segment in people's tab using unique defining attribute and export users selecting all the attributes, click on "select all")
*   Facebook Ads data in xlsx (go to the ad campaign and create a report only using "ad set" as a parameter, don't include "ad name", pay attention to the date range you select in FB Ads, the results might be wrong or different if wrong dates are selected)


**Adding the data to your code**

After you have downloaded all the data to your machine, you will add it to your files folder by clicking on the file icon on the right. Click on the option "Upload to session storage" and upload both sheets here. The 2 data sheets will appear next to "sample data" folder. Next you need to copy the path of each sheet by clicking on 3 dots next to each file name and clicking on "copy path". You will paste this path in the spaces provided in the top portion of this code.

Now you can go through the code and follow the instructions hashed in green.

In [ ]:
# We will install dataprep package (it will take a minute to load)

! pip install dataprep

In [103]:
# From dataprep we will import clean_phone function

from dataprep.clean import clean_phone

In [104]:
# Import librarires for working with our data

import pandas as pd
import numpy as np
from datetime import date

In [ ]:
# Copy the path from Chatfuel data by clicking on 3 dots next to the file name
# Paste this path in the place of the previous path, you will replace the "/content/Localyst_2021_09_28_20_16_04.csv" bit)
# If needed please read the instructions above

cf_data=pd.read_csv("/content/Localyst_2021_10_31_00_57_00.csv")
cf_data.head(5)

In [ ]:
# Copy the path from Facebook Ads data by clicking on 3 dots next to the file name
# Paste this path in the place of the previous path, you will replace the "/content/Virginia_09-28_new.xlsx" bit)
# If needed please read the instructions above

fb_data=pd.read_excel("/content/GA_GOTV_FB_11-30-Oct-1-2021-to-Oct-30-2021.xlsx")
fb_data.head(3)

**If you have succesfully copied and pasted the 3 paths, you can run the whole script.**
To run the script go to "Runtime" in the top menu and click on "Run all".
The script will pause when it will ask you to enter your authentification to mount the drive to place the produced report on it. You will find more instructions at that step.

In [107]:
# set bold style for headers

class style:
   BOLD = '\033[1m'
   END = '\033[0m'

In [ ]:
# Drop first row from the dataframe in Facebook ads data

fb_data.drop([0], inplace = True)
fb_data.head(3)

In [ ]:
# Grab the first row and make it the dataframe header in Facebook Ads data

new_header = fb_data.iloc[0] #grab the first row for the header
fb_data.columns = new_header #set the header row as the df header
fb_data.drop([1], inplace = True) # drop the first row (only use this line if it is an extra text header)
fb_data.head(10)

In [110]:
# Check columns in Facebook Ads data

fb_data.columns

Index([                        nan,               'Ad Set Name',
                               nan,           'Delivery Status',
                  'Delivery Level',             'Campaign Name',
             'Attribution Setting',               'Result Type',
                         'Results',                     'Reach',
                     'Impressions',           'Cost per Result',
                 'Quality Ranking',   'Engagement Rate Ranking',
         'Conversion Rate Ranking',        'Amount Spent (USD)',
       'New Messaging Connections',               'Link Clicks',
                'Reporting Starts',            'Reporting Ends'],
      dtype='object', name=1)

In [111]:
# Rename certian columns in Facebook Ads data to avoid spaces

fb_data = fb_data.rename(columns={'Ad Name':'ad_name'})
fb_data = fb_data.rename(columns={'Campaign Name':'campaign_name'})
fb_data = fb_data.rename(columns={'Ad Set Name':'adset_name'})
fb_data = fb_data.rename(columns={'New Messaging Connections':'new_messaging_connections'})
fb_data = fb_data.rename(columns={'Cost per Result':'cost_per_result'})
fb_data = fb_data.rename(columns={'Amount Spent (USD)':'amount_spent_usd'})
fb_data = fb_data.rename(columns={'Link Clicks':'link_clicks'})
fb_data.columns

Index([                        nan,                'adset_name',
                               nan,           'Delivery Status',
                  'Delivery Level',             'campaign_name',
             'Attribution Setting',               'Result Type',
                         'Results',                     'Reach',
                     'Impressions',           'cost_per_result',
                 'Quality Ranking',   'Engagement Rate Ranking',
         'Conversion Rate Ranking',          'amount_spent_usd',
       'new_messaging_connections',               'link_clicks',
                'Reporting Starts',            'Reporting Ends'],
      dtype='object', name=1)

In [ ]:
# Delete rows in Facebook Ads Data where "results" and "adset_name" column values are NaN (this allows us to delete summary rows)

fb_data = fb_data.dropna(subset=['Results', 'adset_name'])
fb_data

In [113]:
# fb results sum of all adsets

fb_results=fb_data['Results'].sum()
fb_results

2232

In [ ]:
# fb mean of all results

fb_cost_per_result_r=fb_data['cost_per_result'].mean()
fb_cost_per_result='${:0,.2f}'.format(fb_data['cost_per_result'].mean())
fb_cost_per_result

In [ ]:
# alternative way to get cost per result

fb_cost_per_result_alt=fb_data['amount_spent_usd'].sum()/fb_data['Results'].sum()
fb_cost_per_result_alt
fb_cost_per_result_alt_for=fb_cost_per_result='${:0,.2f}'.format(fb_cost_per_result_alt)
fb_cost_per_result_alt_for

In [ ]:
# fb ad spend all adsets

fb_total_ad_spend_r=fb_data['amount_spent_usd'].sum()
fb_total_ad_spend='${:0,.2f}'.format(fb_data['amount_spent_usd'].sum())
fb_total_ad_spend

In [117]:
# fb new messaging conversations started

fb_new_messaging_connections=fb_data['new_messaging_connections'].sum()
fb_new_messaging_connections

2147

In [118]:
# check all unique adset names

fb_data['adset_name'].unique()

array(['georgia_atlanta_gotv_scaled'], dtype=object)

In [119]:
# Save total number of Chatfuel subscribers

total_cf_subscribers=len(cf_data)
total_cf_subscribers

2087

In [120]:
# Check the values for voting status attribute in CF

cf_data['voter_status'].value_counts()

will_vote_georgia_gotv_2021    907
voted_georgia_gotv_2021        207
Name: voter_status, dtype: int64

In [121]:
cf_data['issue'].value_counts()

weed                  2026
women                   13
rent_relief             11
affordable_housing       5
climate                  2
Name: issue, dtype: int64

In [122]:
cf_data['sign_petition'].value_counts()

georgia_gotv_2021                1679
vr_flow_virginia_2021_climate       2
marijuana_2021_localyst_A           1
Name: sign_petition, dtype: int64

In [123]:
# Save total number of petition signatures

sign_petition=sum(cf_data['sign_petition'].value_counts())
sign_petition

1682

In [124]:
# Save total number of phone numbers

phones_cf=sum(cf_data['phone_number'].value_counts())
phones_cf

1083

In [ ]:
# Save a slice of data with phone numbers that are digits (not words)

phone_list=cf_data[cf_data['phone_number'].apply(lambda x: str(x).isdigit())]
phone_list

In [126]:
# Save total number of phone numbers submitted as digits

number_phones_submitted=len(phone_list['phone_number'].value_counts())
number_phones_submitted

862

In [127]:
# we will clean the data using clean_phone
# The results between using clean_phone and the lambda function "isdigit" 2 lines above are slightly different, clean_phone list is larger
# For our metrics we will use the phone numbers from the lambda function (to be on a conservative side)
# For our partners we will extract the subscriber report with clean phone numbers using clean_phone (in hopes that it does a better job caputring more phones)

from dataprep.clean import clean_phone
cf_data=clean_phone(cf_data, 'phone_number')

Phone Number Cleaning Report:
	899 values cleaned (43.08%)
	121 values unable to be parsed (5.8%), set to NaN
Result contains 962 (46.09%) values in the correct format and 1125 null values (53.91%)


In [128]:
# Save total number of frame change agrees
frame_agree=sum(cf_data['frame_change_agreed'].value_counts())
frame_agree

30

In [129]:
# Save total number of share agrees
share_agree=sum(cf_data['share_card_received'].value_counts())
share_agree

469

In [130]:
# Save total number of viral users
viral_users=sum(cf_data['viral_subscriber'].value_counts())
viral_users

30

In [131]:
# Create, format and save all the values we need for our report

# completed=len(ge_rv_data[ge_rv_data['Status'] == 'Complete'])
# started=len(ge_rv_data[ge_rv_data['Status'] != 'Complete'])
# total=completed+started
# cost_per_registered_user='${:0,.2f}'.format(fb_total_ad_spend_r/completed)
# cost_per_registration_started='${:0,.2f}'.format(fb_total_ad_spend_r/total)
# percent_completed='{:.0%}'.format(completed / total)
# cf_rtv_completed='{:.0%}'.format(completed/not_vr_cf)
# cf_rtv_started='{:.0%}'.format(total/not_vr_cf)
# cf_rtv_completed_from_all='{:.0%}'.format(completed/total_cf)

voted_cf=len(cf_data[cf_data['voter_status']== 'voted_georgia_gotv_2021'])
will_vote_cf=len(cf_data[cf_data['voter_status']== 'will_vote_georgia_gotv_2021'])
total_vote_actions=voted_cf+will_vote_cf
percent_voted='{:.0%}'.format(voted_cf/total_vote_actions)
percent_will_vote='{:.0%}'.format(will_vote_cf/total_vote_actions)
cost_voter_action='${:0,.2f}'.format(fb_total_ad_spend_r/total_vote_actions)
cost_per_result_cf='${:0,.2f}'.format(fb_total_ad_spend_r/total_cf_subscribers)
percent_petitions_signed='{:.0%}'.format(sign_petition/total_cf_subscribers)
percent_phones_submitted='{:.0%}'.format(number_phones_submitted/total_cf_subscribers)
percent_voting_actions='{:.0%}'.format(total_vote_actions/total_cf_subscribers)
percent_share_agree='{:.0%}'.format(share_agree/total_cf_subscribers)
percent_frame_agree='{:.0%}'.format(frame_agree/total_cf_subscribers)
percent_virals='{:.0%}'.format(viral_users/total_cf_subscribers)
cost_per_phone='${:0,.2f}'.format(fb_total_ad_spend_r/number_phones_submitted)
cost_per_petition='${:0,.2f}'.format(fb_total_ad_spend_r/sign_petition)

In [ ]:
# initialize list of lists for the report

new_df = [
          ['Total Ad Spend (USD)', fb_total_ad_spend],
          ['Total Subscribers Acquired FB (Results)', fb_results],
          ['Total Subscribers Acquired CF', total_cf_subscribers],
          ['Cost Per Result FB Ads (USD)', fb_cost_per_result_alt_for],
          ['Cost Per Acquisition CF', cost_per_result_cf],
          ['Total Petitions Signed', sign_petition],
          ['% of Petitions Signed', percent_petitions_signed],
          ['Cost Per Petition Signed', cost_per_petition],
          ['Total Phone Numbers Opted In', number_phones_submitted],
          ['% Phone Numbers Opted In', percent_phones_submitted],
          ['Cost Per Phone Number', cost_per_phone],
          ['Voting Response Engagements', total_vote_actions],
          ['% Voting Response Engagements From All CF Users', percent_voting_actions],
          ['Already Voted From Engaged', voted_cf],
          ['% of Already Voted', percent_voted],
          ['Will Vote', will_vote_cf],
          ['% of Will Vote From Engaged', percent_will_vote],
          ['Cost Per Voting Action', cost_voter_action],
          ['Total Agree to Relational Shares', share_agree],
          ['% of Total Agree to Relational Shares', percent_share_agree],
          ['Total Frame Change Agree', frame_agree],
          ['% of Total Frame Change Agree', percent_frame_agree],
          ['Viral Subscribers', viral_users],
          ['% of Total Viral Subscribers', percent_virals]
          
          
          
        ]
 
# Create the pandas DataFrame
gotv_metrics = pd.DataFrame(new_df, columns = ['Description', 'Metric'])
gotv_metrics

In [ ]:
# EDA: subgroup by social issue

cf_data_issue = cf_data.groupby(['issue', 'voter_status'])
cf_data_issue.first()

In [134]:
# EDA: voter status by issue

cf_issue = cf_data[['issue', 'voter_status']]
breakdown_issue_cf=cf_issue.value_counts()
breakdown_issue_cf

issue               voter_status               
weed                will_vote_georgia_gotv_2021    895
                    voted_georgia_gotv_2021        204
rent_relief         will_vote_georgia_gotv_2021      5
women               will_vote_georgia_gotv_2021      4
rent_relief         voted_georgia_gotv_2021          2
affordable_housing  will_vote_georgia_gotv_2021      2
climate             will_vote_georgia_gotv_2021      1
affordable_housing  voted_georgia_gotv_2021          1
dtype: int64

In [135]:
# EDA: petition signature by issue

cf_issue = cf_data[['issue', 'sign_petition']]
cf_issue.value_counts()

issue               sign_petition                
weed                georgia_gotv_2021                1657
women               georgia_gotv_2021                   8
rent_relief         georgia_gotv_2021                   8
affordable_housing  georgia_gotv_2021                   4
weed                vr_flow_virginia_2021_climate       2
climate             georgia_gotv_2021                   2
weed                marijuana_2021_localyst_A           1
dtype: int64

In [136]:
from datetime import datetime
from pytz import timezone
import pytz

def get_pst_time():
    date_format='%m_%d_%Y_%H_%M_%S_%Z'
    date = datetime.now(tz=pytz.utc)
    date = date.astimezone(timezone('US/Pacific'))
    pstDateTime=date.strftime(date_format)
    return pstDateTime


date_PDT=get_pst_time()
date_PDT

'10_30_2021_18_39_22_PDT'

In [137]:
today = date.today()
today = today.strftime("%b-%d-%Y")
today

'Oct-31-2021'

**Here you need to click on the link and it will create a authentification token that you will plaste in the space provided in the code**

Click on the link that appears after "Go to this URL in a browser:"
Chose your google account, sign in, copy the token.
Paste the token in the slot provided in the script and click "enter".

In [138]:
# Mount drive from google

from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [139]:
# Export metrics report back to files
# The files will be in the same folder where you uploaded the data

gotv_metrics.to_csv(f'{date_PDT}-Early Vote GOTV Metrics Georgia.csv')
!cp Early_Vote_GOTV_Metrics_Georgia.csv "drive/My Drive/"

cp: cannot stat 'Early_Vote_GOTV_Metrics_Georgia.csv': No such file or directory


Next step is to get a list of cleaned phone numbers for our partners

In [ ]:
# We will create a new dataframe with 6 parameters for our partners

# We will create a new dataframe with 6 parameters for our partners

clean_phone_data = cf_data.dropna(subset=['phone_number_clean'])
subscribers_phone_numbers = clean_phone_data[['profile pic url','first name', 'last name','phone_number_clean', 'voter_status', 'issue' ]]
subscribers_phone_numbers

In [141]:
# Export metrics report back to files
# The files will be in the same folder where you uploaded the data

subscribers_phone_numbers.to_csv(f'{date_PDT}-Phone_Numbers_Georgia_Subscribers.csv')
!cp Phone_Numbers_Georgia_Subscribers.csv "drive/My Drive/"

cp: cannot stat 'Phone_Numbers_Georgia_Subscribers.csv': No such file or directory
